In [1]:
import mta_functions as mta
import pandas as pd
import eda

In [2]:
# Create dataframe from unprocessed directory files
mta_data = mta.df_from_directory('../data/turnstiles/processing')

In [3]:
# Create concat string for turnstile
mta_data['TID'] = mta.concat_columns(mta_data, ['C/A', 'UNIT', 'SCP'])

In [4]:
# Create Turnstile ID for each turnstile
mta_data['TID'] = mta_data.apply(lambda x: mta.create_hashed_column(x['TID'], 16), axis = 1)

In [5]:
# Create concat string for entries
mta_data['UID_string'] = mta.concat_columns(mta_data, ['TID', 'DATE', 'TIME'])

In [6]:
# Convert entry string to hash
mta_data['UID'] = mta_data.apply(lambda x: mta.create_hashed_column(x['UID_string'], 16), axis = 1)

In [7]:
# Create DateTime column
mta_data['DATETIME'] = pd.to_datetime(mta_data.DATE + ' ' + mta_data.TIME)

In [8]:
#  Fix EXITS column name
mta_data.columns = [column.strip() for column in mta_data.columns]

In [9]:
# Reorganize Columns
mta_data = mta_data[['UID','TID','C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATETIME','DESC','ENTRIES','EXITS']]

In [10]:
# mta_data.loc[mta_data['DESC'] == 'RECOVR AUD'].index

# idx = mta_data.index.iloc(row_index)
# mta_data.iloc[idx - 1 : idx + 1]

# list(mta_data[mta_data.DESC == 'RECOVR AUD'].ind)

# mta_data.index.

In [11]:
mta_data.describe().apply(lambda s: s.apply('{0:.1f}'.format))

,ENTRIES,EXITS
count,3295462.0,3295462.0
mean,42830005.3,35090932.2
std,218833245.6,198751684.2
min,0.0,0.0
25%,317884.0,141535.2
50%,1921470.0,1124620.0
75%,6572069.0,4477027.0
max,2128895656.0,2123501742.0


In [12]:
eda.identify_nulls(mta_data)

------------------Rows with Nulls------------------


,UID,TID,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,DESC,ENTRIES,EXITS


In [13]:
eda.identify_duplicates(mta_data, 'UID', 'UNIT')

--------------Possible Duplicate Rows--------------


,UID,TID,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,DESC,ENTRIES,EXITS
184505,024E4A629B63F671,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-21 07:00:00,REGULAR,17,0
184506,024E4A629B63F671,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-21 07:00:00,RECOVR AUD,15,0
184515,0922F21EDC911A87,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-22 03:00:00,REGULAR,15,0
184516,0922F21EDC911A87,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-22 03:00:00,RECOVR AUD,17,0
184499,0A2E91A008B6CEC5,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-20 19:00:00,REGULAR,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...
184492,F72F8DE19515A966,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-20 03:00:00,RECOVR AUD,15,0
184503,F8C9004FC73D9CD3,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-21 03:00:00,REGULAR,17,0
184504,F8C9004FC73D9CD3,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-21 03:00:00,RECOVR AUD,15,0
184483,FB3B3F0E08207E89,B4CD039EDBAFDB6A,R514,R094,00-05-00,ASTORIA BLVD,NQW,BMT,2020-01-19 11:00:00,REGULAR,15,0


In [14]:
mta_data[mta_data.UID == '78B40D7CD2F90B1A']

,UID,TID,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,DESC,ENTRIES,EXITS
32400,78B40D7CD2F90B1A,892D72F2533CCDC3,H009,R235,00-03-00,BEDFORD AV,L,BMT,2020-03-22 12:00:00,REGULAR,17126296,22672798
32401,78B40D7CD2F90B1A,892D72F2533CCDC3,H009,R235,00-03-00,BEDFORD AV,L,BMT,2020-03-22 12:00:00,RECOVR AUD,19676995,30044802


In [15]:
# mta_data.sort_values(["UID", "DESC"], inplace=True, ascending=False)
# mta_data.drop_duplicates(subset=["UID"], inplace=True, keep='first')

In [16]:
# get names of indexes for which
# column Age has value 21
mta_data.drop(mta_data[mta_data['DESC'] == 'RECOVR AUD'].index, inplace = True)

In [17]:
# Create Weekday from newly create DateTime column
mta_data['DoW'] = mta_data['DATETIME'].dt.day_name()

In [18]:
# Identify Daytime
mta_data['WEEKEND'] = mta_data['DoW'].apply(lambda x: True if (x == 'Saturday' or x == 'Sunday') else False)

In [19]:
# Create Time of Day
mta_data['HOUR'] = mta_data['DATETIME'].dt.hour

In [20]:
# Daily Counts by Turnstile
mta_data['ENTRYDIFF'] = mta_data['ENTRIES'].diff()
mta_data['EXITDIFF'] = mta_data['EXITS'].diff()

In [21]:
mta_data['DURATION'] = mta_data.DATETIME.diff().apply(lambda x: x.total_seconds()/3600)

In [22]:
# Drop Rows without changes
mta_data.dropna(subset=['ENTRYDIFF', 'EXITDIFF'], inplace=True)

In [23]:
# Drop First Entry Per Turnstile
# mta_data = mta_data[mta_data.DATETIME >= mta_data.DATETIME.dt.day.min() + pd.DateOffset(days=1)]

In [24]:
# Normalize floats to int values
mta_data['ENTRYDIFF'] = mta_data['ENTRYDIFF'].astype(int)
mta_data['EXITDIFF'] = mta_data['EXITDIFF'].astype(int)
mta_data['TOTAL_TRAFFIC'] = mta_data['ENTRYDIFF'] + mta_data['EXITDIFF']

In [25]:
mta_data['AVG_TRAFFIC'] = mta_data['TOTAL_TRAFFIC'] / mta_data['DURATION']

In [26]:
# Drop ENTRYDIFF AND EXITDIFF Rows with extreme values
mta_data = mta_data[mta_data.ENTRYDIFF <= 10000]
mta_data = mta_data[mta_data.ENTRYDIFF > 0]

mta_data = mta_data[mta_data.EXITDIFF <= 10000]
mta_data = mta_data[mta_data.EXITDIFF > 0]

In [27]:
# Restructure columns
columns_order = ['TID','UNIT','SCP','STATION','LINENAME','DIVISION','DATETIME','HOUR','DoW','WEEKEND','DESC','ENTRIES','EXITS','ENTRYDIFF','EXITDIFF','DURATION','TOTAL_TRAFFIC','AVG_TRAFFIC']
mta_data = mta_data[columns_order]

In [28]:
mta_data.head(20)

,TID,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,HOUR,DoW,WEEKEND,DESC,ENTRIES,EXITS,ENTRYDIFF,EXITDIFF,DURATION,TOTAL_TRAFFIC,AVG_TRAFFIC
5,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-07 23:00:00,23,Saturday,True,REGULAR,7403197,2512159,157,27,4.0,184,46.000000
6,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-08 04:00:00,4,Sunday,True,REGULAR,7403226,2512176,29,17,5.0,46,9.200000
7,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-08 08:00:00,8,Sunday,True,REGULAR,7403240,2512195,14,19,4.0,33,8.250000
8,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-08 12:00:00,12,Sunday,True,REGULAR,7403299,2512245,59,50,4.0,109,27.250000
10,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-08 20:00:00,20,Sunday,True,REGULAR,7403667,2512332,368,87,8.0,455,56.875000
11,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-09 00:00:00,0,Monday,False,REGULAR,7403756,2512351,89,19,4.0,108,27.000000
12,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-09 04:00:00,4,Monday,False,REGULAR,7403762,2512352,6,1,4.0,7,1.750000
13,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-09 08:00:00,8,Monday,False,REGULAR,7403784,2512463,22,111,4.0,133,33.250000
14,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-09 12:00:00,12,Monday,False,REGULAR,7403915,2512689,131,226,4.0,357,89.250000
15,90191926F891F8C7,R051,02-00-00,59 ST,NQR456W,BMT,2020-03-09 16:00:00,16,Monday,False,REGULAR,7404153,2512730,238,41,4.0,279,69.750000


In [29]:
dupes = eda.identify_duplicates(mta_data,'TID', 'ENTRIES')
dupes.sort_values(by=['TID','DATETIME'], ascending=True)

--------------Possible Duplicate Rows--------------


,TID,UNIT,SCP,STATION,LINENAME,DIVISION,DATETIME,HOUR,DoW,WEEKEND,DESC,ENTRIES,EXITS,ENTRYDIFF,EXITDIFF,DURATION,TOTAL_TRAFFIC,AVG_TRAFFIC
151729,081CADC137B8554A,R194,00-00-00,BLEECKER ST,6DF,IRT,2020-03-09 12:00:00,12,Monday,False,REGULAR,536805,173160,1,2,0.015000,3,200.000000
151731,081CADC137B8554A,R194,00-00-00,BLEECKER ST,6DF,IRT,2020-03-09 12:01:47,12,Monday,False,REGULAR,536805,173160,1,2,0.022500,3,133.333333
115619,085E8E2F31111820,R549,00-01-04,NEWARK HW BMEBE,1,PTH,2019-12-28 03:04:20,3,Saturday,True,REGULAR,304,174783,113,2989,-163.313611,3102,-18.994130
114785,085E8E2F31111820,R549,00-01-04,NEWARK HW BMEBE,1,PTH,2020-01-11 03:04:20,3,Saturday,True,REGULAR,304,184812,113,186,-163.222500,299,-1.831855
114803,085E8E2F31111820,R549,00-01-04,NEWARK HW BMEBE,1,PTH,2020-01-14 06:40:20,6,Tuesday,False,REGULAR,307,186301,2,19,4.200000,21,5.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96030,FD7E05AB4863CA8C,R020,01-06-01,47-50 STS ROCK,BDFM,IND,2020-02-08 19:00:00,19,Saturday,True,REGULAR,589895,38,43,11,4.000000,54,13.500000
96031,FD7E05AB4863CA8C,R020,01-06-01,47-50 STS ROCK,BDFM,IND,2020-02-08 23:00:00,23,Saturday,True,REGULAR,589914,43,19,5,4.000000,24,6.000000
96040,FD7E05AB4863CA8C,R020,01-06-01,47-50 STS ROCK,BDFM,IND,2020-02-10 11:00:00,11,Monday,False,REGULAR,590033,544,41,461,4.000000,502,125.500000
121658,FFA87F8C5782C55C,R542,00-00-02,TWENTY THIRD ST,1,PTH,2020-01-02 09:51:07,9,Thursday,False,REGULAR,98269,69469,228,2,4.200000,230,54.761905


In [30]:
mta_data.describe().apply(lambda s: s.apply('{0:.1f}'.format))

,HOUR,ENTRIES,EXITS,ENTRYDIFF,EXITDIFF,DURATION,TOTAL_TRAFFIC,AVG_TRAFFIC
count,2380517.0,2380517.0,2380517.0,2380517.0,2380517.0,2380517.0,2380517.0,2380517.0
mean,11.8,29224350.2,21401154.4,153.2,125.5,4.2,278.8,66.0
std,6.8,169592003.0,138851891.5,252.8,236.5,2.8,403.8,86.8
min,0.0,1.0,1.0,1.0,1.0,-166.9,2.0,-1680.9
25%,7.0,579408.0,354455.0,15.0,13.0,4.0,38.0,9.5
50%,12.0,2619817.0,1594970.0,58.0,45.0,4.0,129.0,31.5
75%,17.0,7028238.0,5021985.0,190.0,137.0,4.0,368.0,89.5
max,23.0,2116079989.0,2038505334.0,9832.0,9949.0,148.0,19282.0,2290.9
